<a href="https://colab.research.google.com/github/seobando/UDACITY_GenerativeAI/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q "datasets==2.15.0"

In [ ]:
#!pip install -U transformer

In [ ]:
!pip install peft

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: gpt2
* Evaluation approach: accuracy evaluation
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
dataset_name= "imdb"
model_name = "gpt2"

In [ ]:
# Import the datasets and transformers packages

from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset(dataset_name, split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(800))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 800
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 800
 })}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    """Preprocess the emotion dataset by returning tokenized examples."""
    return tokenizer(
        examples["text"],
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=128
    )

In [ ]:
tokenized_data = {}

for split in splits:
    tokenized_data[split] = ds[split].map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

model.config.pad_token_id = tokenizer.pad_token_id

for param in model.base_model.parameters():
    param.requires_grad = False

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer, DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args= training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.706763,0.600000


Checkpoint destination directory ./results/checkpoint-200 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=200, training_loss=0.7349045562744141, metrics={'train_runtime': 148.3471, 'train_samples_per_second': 5.393, 'train_steps_per_second': 1.348, 'total_flos': 418074800947200.0, 'train_loss': 0.7349045562744141, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7067633271217346,
 'eval_accuracy': 0.6,
 'eval_runtime': 69.4295,
 'eval_samples_per_second': 11.522,
 'eval_steps_per_second': 2.881,
 'epoch': 1.0}

In [ ]:
trainer.save_model("gpt2_model")

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
gpt_model = AutoModelForSequenceClassification.from_pretrained("gpt2_model")

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    inference_mode=False,
    r=18,
    lora_alpha=32,
    lora_dropout=0.1,
    modules_to_save=["classifier"],
    bias='lora_only',
)

lora_model = get_peft_model(gpt_model, config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['c_attn', 'c_proj'],
    lora_dropout=0.1,
    bias="none",
    fan_in_fan_out=True,
    task_type=TaskType.SEQ_CLS
)

lora_model = get_peft_model(model, config)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    label_names=["labels"],
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.643052,0.638750
2,No log,0.581161,0.716250
3,No log,0.553598,0.737500


Checkpoint destination directory ./results/checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-200 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-300 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=300, training_loss=0.5003304036458334, metrics={'train_runtime': 72.4395, 'train_samples_per_second': 33.131, 'train_steps_per_second': 4.141, 'total_flos': 158275731456000.0, 'train_loss': 0.5003304036458334, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5535978078842163,
 'eval_accuracy': 0.7375,
 'eval_runtime': 6.9416,
 'eval_samples_per_second': 115.248,
 'eval_steps_per_second': 14.406,
 'epoch': 3.0}

In [ ]:
lora_model.save_pretrained("gpt-custom-lora")

In [ ]:
tokenizer.save_pretrained("gpt-custom-lora-tokenize")

('gpt-custom-lora-tokenize/tokenizer_config.json',
 'gpt-custom-lora-tokenize/special_tokens_map.json',
 'gpt-custom-lora-tokenize/vocab.json',
 'gpt-custom-lora-tokenize/merges.txt',
 'gpt-custom-lora-tokenize/added_tokens.json',
 'gpt-custom-lora-tokenize/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [59]:
from peft import AutoPeftModelForSequenceClassification

lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-custom-lora").to(device)

#tokenizer = AutoTokenizer.from_pretrained("gpt2_lora_tokenizer")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
import torch

# Define input text
input_text = [
    "I absolutely loved the cinematography and the performances in this film. It's a masterpiece!",
    "The plot was convoluted, and the acting felt forced. I wouldn't recommend this movie to anyone."
]

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
input_ids= inputs['input_ids']
attention_mask = inputs['attention_mask']
# Perform inference
with torch.no_grad():
    outputs = lora_model(torch.tensor(input_ids, device=device), attention_mask=torch.tensor(attention_mask, device=device))

# Convert logits to probabilities
probs = torch.softmax(outputs.logits, dim=1)

# Get predicted labelsk
predicted_labels = torch.argmax(probs, dim=1)

# Map labels to sentiments
sentiments = ["NEGATIVE", "POSITIVE"]

# Print results
for i, text in enumerate(input_text):
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {sentiments[predicted_labels[i].item()]} (Probability: {probs[i][predicted_labels[i]].item()})")


<ipython-input-66-e0ef6973f37d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = lora_model(torch.tensor(input_ids, device=device), attention_mask=torch.tensor(attention_mask, device=device))


AssertionError: Cannot handle batch sizes > 1 if no padding token is defined.